### Final version with profile management and matchmaking tabs in UI

The KBS is designed to connect users based on shared preferences and compatibility factors. Its UI features two main tabs:

- Profile Management Tab: Allows users to create, update, and manage their profiles. Users can also adjust the importance of matchmaking attributes (age, interests, location, education, occupation, and lifestyle) using weight sliders.

- Matchmaking Tab: Uses the user’s profile and custom weights to find the most compatible matches, ranking them based on a dynamic scoring system.

In [2]:
pip install ipywidgets


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Before executing, please install ipywidgets using command: pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output

class KnowledgeBase:
    def __init__(self):
        self.profiles = {}
        self.next_id = 1

    def add_profile(self, profile_data):
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}
        self.next_id += 1
        return profile_id

    def update_profile(self, profile_id, updated_data):
        if profile_id in self.profiles:
            self.profiles[profile_id].update(updated_data)
            return True
        return False

    def get_profile(self, profile_id):
        return self.profiles.get(profile_id)

    def get_all_profiles(self):
        return list(self.profiles.values())


class Matchmaker:
    def __init__(self, knowledge_base, max_age_diff=10):
        self.knowledge_base = knowledge_base
        self.max_age_diff = max_age_diff

    def calculate_compatibility(self, profile_a, profile_b):
        if profile_a['preferred_gender'] not in [profile_b['gender'], "Any"]:
            return 0, []
        if profile_b['preferred_gender'] not in [profile_a['gender'], "Any"]:
            return 0, []

        matched = []
        custom_weights = profile_a.get("weights", {
            'age': 0.2, 'interests': 0.3, 'location': 0.15,
            'education': 0.1, 'occupation': 0.1, 'lifestyle': 0.15
        })

        # Age Match
        age_diff = abs(profile_a['age'] - profile_b['age'])
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * custom_weights['age']
        if age_score > 0:
            matched.append("Age")

        # Interests Match
        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = len(interests_a & interests_b)
        union = len(interests_a | interests_b)
        jaccard_score = (intersection / union) * custom_weights['interests'] if union > 0 else 0
        if intersection > 0:
            matched.append(f"Interests: {', '.join(interests_a & interests_b)}")

        # Location Match
        location_score = custom_weights['location'] if profile_a['location'] == profile_b['location'] else 0
        if location_score > 0:
            matched.append("Location")

        # Education Match
        education_score = custom_weights['education'] if profile_a['education'] == profile_b['education'] else 0
        if education_score > 0:
            matched.append("Education")

        # Occupation Match
        occupation_score = custom_weights['occupation'] if profile_a['occupation'] == profile_b['occupation'] else 0
        if occupation_score > 0:
            matched.append("Occupation")

        # Lifestyle Match
        lifestyle_score = custom_weights['lifestyle'] if profile_a['lifestyle'] == profile_b['lifestyle'] else 0
        if lifestyle_score > 0:
            matched.append("Lifestyle")

        # Total Score
        total_score = (
            age_score + jaccard_score + location_score +
            education_score + occupation_score + lifestyle_score
        )
        return total_score, matched

    def find_matches(self, user_id, top_n=5):
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue
            score, details = self.calculate_compatibility(target_profile, profile)
            if score > 0:
                matches.append((profile['id'], profile['name'], round(score, 2), details))

        matches.sort(key=lambda x: x[2], reverse=True)
        return matches[:top_n]


# Initialize KnowledgeBase and Matchmaker
kb = KnowledgeBase()
matchmaker = Matchmaker(knowledge_base=kb)

# 30 samples with different attributes and gender preferences

sample_profiles = [
 
    {'name': 'Brian', 'age': 24, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Nairobi',
     'interests': ['football', 'hiking', 'coding'], 'education': 'Bachelor', 'occupation': 'Software Developer',
     'relationship_goals': 'Long-term', 'lifestyle': 'Non-Smoker'},

    {'name': 'Emily', 'age': 27, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Mombasa',
     'interests': ['travel', 'beach walks', 'reading'], 'education': 'Master', 'occupation': 'Lecturer',
     'relationship_goals': 'Marriage', 'lifestyle': 'Active'},

    {'name': 'James', 'age': 30, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Kisumu',
     'interests': ['fishing', 'music', 'cycling'], 'education': 'Bachelor', 'occupation': 'Doctor',
     'relationship_goals': 'Marriage', 'lifestyle': 'Active'},

    {'name': 'Linda', 'age': 25, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Eldoret',
     'interests': ['fashion', 'photography', 'cooking'], 'education': 'Bachelor', 'occupation': 'Journalist',
     'relationship_goals': 'Casual', 'lifestyle': 'Non-Smoker'},

    {'name': 'Michael', 'age': 35, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Nakuru',
     'interests': ['farming', 'politics', 'hiking'], 'education': 'PhD', 'occupation': 'Agricultural Economist',
     'relationship_goals': 'Long-term', 'lifestyle': 'Non-Smoker'},

    {'name': 'Catherine', 'age': 26, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Thika',
     'interests': ['business', 'reading', 'coffee dates'], 'education': 'Bachelor', 'occupation': 'Entrepreneur',
     'relationship_goals': 'Long-term', 'lifestyle': 'Smoker'},

    {'name': 'David', 'age': 31, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Machakos',
     'interests': ['hiking', 'biking', 'camping'], 'education': 'Master', 'occupation': 'Civil Engineer',
     'relationship_goals': 'Marriage', 'lifestyle': 'Active'},

    {'name': 'Grace', 'age': 29, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Nyeri',
     'interests': ['cooking', 'gardening', 'travel'], 'education': 'Bachelor', 'occupation': 'Nurse',
     'relationship_goals': 'Marriage', 'lifestyle': 'Non-Smoker'},

    {'name': 'Joshua', 'age': 40, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Meru',
     'interests': ['farming', 'wildlife conservation', 'hiking'], 'education': 'Bachelor', 'occupation': 'Wildlife Ranger',
     'relationship_goals': 'Long-term', 'lifestyle': 'Active'},

    {'name': 'Beatrice', 'age': 25, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Kakamega',
     'interests': ['dance', 'music', 'social work'], 'education': 'Master', 'occupation': 'Social Worker',
     'relationship_goals': 'Casual', 'lifestyle': 'Non-Smoker'},

    {'name': 'Alex', 'age': 29, 'gender': 'Non-Binary', 'preferred_gender': 'Any', 'location': 'Kisumu',
     'interests': ['art', 'poetry', 'philosophy'], 'education': 'Bachelor', 'occupation': 'Designer',
     'relationship_goals': 'Long-term', 'lifestyle': 'Non-Smoker'},

    {'name': 'Mark', 'age': 26, 'gender': 'Male', 'preferred_gender': 'Male', 'location': 'Nairobi',
     'interests': ['gaming', 'music', 'movies'], 'education': 'Bachelor', 'occupation': 'Software Engineer',
     'relationship_goals': 'Marriage', 'lifestyle': 'Active'},

    {'name': 'Sophia', 'age': 30, 'gender': 'Female', 'preferred_gender': 'Female', 'location': 'Mombasa',
     'interests': ['yoga', 'meditation', 'gardening'], 'education': 'PhD', 'occupation': 'Psychologist',
     'relationship_goals': 'Long-term', 'lifestyle': 'Non-Smoker'},

    {'name': 'Eric', 'age': 32, 'gender': 'Male', 'preferred_gender': 'Male', 'location': 'Nakuru',
     'interests': ['swimming', 'reading', 'chess'], 'education': 'Master', 'occupation': 'Architect',
     'relationship_goals': 'Long-term', 'lifestyle': 'Active'},

    {'name': 'Naomi', 'age': 28, 'gender': 'Female', 'preferred_gender': 'Female', 'location': 'Kisumu',
     'interests': ['running', 'fitness', 'nutrition'], 'education': 'Bachelor', 'occupation': 'Personal Trainer',
     'relationship_goals': 'Casual', 'lifestyle': 'Active'},

    {'name': 'Derrick', 'age': 35, 'gender': 'Male', 'preferred_gender': 'Male', 'location': 'Nyeri',
     'interests': ['mountain climbing', 'history', 'travel'], 'education': 'PhD', 'occupation': 'Historian',
     'relationship_goals': 'Marriage', 'lifestyle': 'Non-Smoker'},

    {'name': 'Liam', 'age': 22, 'gender': 'Male', 'preferred_gender': 'Male', 'location': 'Thika',
     'interests': ['photography', 'basketball', 'music'], 'education': 'Bachelor', 'occupation': 'Photographer',
     'relationship_goals': 'Long-term', 'lifestyle': 'Active'},

    {'name': 'Isabel', 'age': 27, 'gender': 'Female', 'preferred_gender': 'Non-Binary', 'location': 'Machakos',
     'interests': ['hiking', 'reading', 'volunteering'], 'education': 'Master', 'occupation': 'Social Worker',
     'relationship_goals': 'Long-term', 'lifestyle': 'Non-Smoker'},

    {'name': 'Chris', 'age': 30, 'gender': 'Non-Binary', 'preferred_gender': 'Any', 'location': 'Nairobi',
     'interests': ['coding', 'board games', 'philosophy'], 'education': 'PhD', 'occupation': 'Researcher',
     'relationship_goals': 'Marriage', 'lifestyle': 'Non-Smoker'},
]

# Add all profiles to the knowledge base
for p in sample_profiles:
    kb.add_profile(p)

# === UI Elements ===
# Profile Inputs
name_input = widgets.Text(description="Name")
age_input = widgets.IntSlider(description="Age", min=18, max=60, value=25)
gender_input = widgets.Dropdown(description="Gender", options=['Male', 'Female', 'Non-Binary'])
preferred_gender_input = widgets.Dropdown(description="Preferred Gender", options=['Male', 'Female', 'Any'])
location_input = widgets.Text(description="Location")
interests_input = widgets.Text(description="Interests (comma-separated)")
education_input = widgets.Dropdown(description="Education", options=['High School', 'Bachelor', 'Master', 'PhD'])
occupation_input = widgets.Text(description="Occupation")
relationship_goals_input = widgets.Dropdown(description="Goals", options=['Casual', 'Long-term', 'Marriage'])
lifestyle_input = widgets.Dropdown(description="Lifestyle", options=['Active', 'Non-Smoker', 'Smoker'])
create_button = widgets.Button(description="Create Profile", button_style="success")
update_button = widgets.Button(description="Update Profile", button_style="info")
profile_output = widgets.Output()

# Weight Adjustment Sliders (Including Location Weight)
age_weight = widgets.FloatSlider(description="Age", min=0, max=1, step=0.05, value=0.2)
interests_weight = widgets.FloatSlider(description="Interests", min=0, max=1, step=0.05, value=0.3)
location_weight = widgets.FloatSlider(description="Location", min=0, max=1, step=0.05, value=0.15)
education_weight = widgets.FloatSlider(description="Education", min=0, max=1, step=0.05, value=0.1)
occupation_weight = widgets.FloatSlider(description="Occupation", min=0, max=1, step=0.05, value=0.1)
lifestyle_weight = widgets.FloatSlider(description="Lifestyle", min=0, max=1, step=0.05, value=0.15)
set_weights_button = widgets.Button(description="Save Weights", button_style="success")
weight_output = widgets.Output()

# Matchmaking UI
matchmaking_button = widgets.Button(description="Find Matches", button_style="primary")
matchmaking_display = widgets.Output()

# Store logged-in user ID
logged_in_user_id = None


# === Profile Management Functions ===
def create_or_update_profile(b):
    global logged_in_user_id
    with profile_output:
        clear_output()
        profile = {
            'name': name_input.value,
            'age': age_input.value,
            'gender': gender_input.value,
            'preferred_gender': preferred_gender_input.value,
            'location': location_input.value,
            'interests': [i.strip() for i in interests_input.value.split(',') if i.strip()],
            'education': education_input.value,
            'occupation': occupation_input.value,
            'relationship_goals': relationship_goals_input.value,
            'lifestyle': lifestyle_input.value
        }

        if logged_in_user_id is None:
            logged_in_user_id = kb.add_profile(profile)
            print(f"✅ Profile created! Your Profile ID is {logged_in_user_id}.")
        else:
            kb.update_profile(logged_in_user_id, profile)
            print(f"🔄 Profile updated successfully.")


def save_custom_weights(b):
    global logged_in_user_id
    with weight_output:
        clear_output()
        if logged_in_user_id is None:
            print("⚠️ No profile found. Create a profile first.")
            return

        # Store weights in the user profile
        kb.get_profile(logged_in_user_id)['weights'] = {
            'age': age_weight.value,
            'interests': interests_weight.value,
            'location': location_weight.value,
            'education': education_weight.value,
            'occupation': occupation_weight.value,
            'lifestyle': lifestyle_weight.value
        }
        print("✅ Weights updated successfully!")


def find_matches(b):
    if logged_in_user_id is None:
        with matchmaking_display:
            clear_output()
            print("⚠️ No profile found. Please create your profile first.")
        return

    with matchmaking_display:
        clear_output()
        matches = matchmaker.find_matches(logged_in_user_id, top_n=3)
        if not matches:
            print("❌ No compatible matches found.")
        else:
            print("🔍 Based on your profile, these are your best matches:\n")
            for profile_id, name, score, details in matches:
                print(f"👤 {name} (Profile {profile_id}) - Compatibility Score: {score}")
                print(f"✅ Matched Attributes: {', '.join(details)}")
                print("-" * 40)


# === UI Setup ===
set_weights_button.on_click(save_custom_weights)
create_button.on_click(create_or_update_profile)
update_button.on_click(create_or_update_profile)
matchmaking_button.on_click(find_matches)

# Profile Management Tab
profile_management = widgets.VBox([
    widgets.Label("📝 Manage Your Profile"),
    name_input, age_input, gender_input, preferred_gender_input, location_input,
    interests_input, education_input, occupation_input, relationship_goals_input, lifestyle_input,
    create_button, update_button, profile_output,
    widgets.Label("⚖️ Adjust Matchmaking Weights (Modify the importance of each attribute to fine-tune matches)"),
    age_weight, interests_weight, location_weight, education_weight, occupation_weight, lifestyle_weight,
    set_weights_button, weight_output
])

# Matchmaking Tab
matchmaking_section = widgets.VBox([
    widgets.Label("💘 Find Your Matches"),
    matchmaking_button, matchmaking_display
])

# Tabs
tab = widgets.Tab()
tab.children = [profile_management, matchmaking_section]
tab.set_title(0, "Profile Management")
tab.set_title(1, "Matchmaking")

display(tab)


### Second version with basic ipywidgets UI for taking inputs

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class KnowledgeBase:
    """
    Stores and manages user profiles with unique identifiers.
    """
    def __init__(self):
        self.profiles = {}
        self.next_id = 1

    def add_profile(self, profile_data):
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}
        self.next_id += 1
        return profile_id

    def get_profile(self, profile_id):
        return self.profiles.get(profile_id)

    def get_all_profiles(self):
        return list(self.profiles.values())


class Matchmaker:
    """
    Implements Tinder-like matchmaking using compatibility rules and weighted attributes.
    """
    def __init__(self, knowledge_base, weights=None, max_age_diff=10):
        self.knowledge_base = knowledge_base
        self.weights = weights or {
            'age': 0.2, 'interests': 0.3, 'location': 0.15,
            'education': 0.1, 'occupation': 0.1, 'lifestyle': 0.15
        }
        self.max_age_diff = max_age_diff

    def calculate_compatibility(self, profile_a, profile_b):
        if profile_a['preferred_gender'] not in [profile_b['gender'], "Any"]:
            return 0, []
        if profile_b['preferred_gender'] not in [profile_a['gender'], "Any"]:
            return 0, []

        matched = []
        age_diff = abs(profile_a['age'] - profile_b['age'])
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * self.weights['age']
        if age_score > 0:
            matched.append("Age")

        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = len(interests_a & interests_b)
        union = len(interests_a | interests_b)
        jaccard_score = (intersection / union) * self.weights['interests'] if union > 0 else 0
        if intersection > 0:
            matched.append(f"Interests: {', '.join(interests_a & interests_b)}")

        location_score = self.weights['location'] if profile_a['location'] == profile_b['location'] else 0
        if location_score > 0:
            matched.append("Location")

        education_score = self.weights['education'] if profile_a['education'] == profile_b['education'] else 0
        if education_score > 0:
            matched.append("Education")

        occupation_score = self.weights['occupation'] if profile_a['occupation'] == profile_b['occupation'] else 0
        if occupation_score > 0:
            matched.append("Occupation")

        lifestyle_score = self.weights['lifestyle'] if profile_a['lifestyle'] == profile_b['lifestyle'] else 0
        if lifestyle_score > 0:
            matched.append("Lifestyle")

        total_score = age_score + jaccard_score + location_score + education_score + occupation_score + lifestyle_score
        return total_score, matched

    def find_matches(self, user_id, top_n=5):
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue
            score, details = self.calculate_compatibility(target_profile, profile)
            if score > 0:
                matches.append((profile['id'], round(score, 2), details))

        matches.sort(key=lambda x: x[1], reverse=True)
        return matches[:top_n]


kb = KnowledgeBase()
sample_profiles = [
    {'age': 24, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Nairobi',
     'interests': ['football', 'hiking', 'coding'], 'education': 'Bachelor', 'occupation': 'Software Developer',
     'relationship_goals': 'Long-term', 'lifestyle': 'Non-Smoker'},

    {'age': 27, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Mombasa',
     'interests': ['travel', 'beach walks', 'reading'], 'education': 'Master', 'occupation': 'Lecturer',
     'relationship_goals': 'Marriage', 'lifestyle': 'Active'},

    {'age': 30, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Kisumu',
     'interests': ['fishing', 'music', 'cycling'], 'education': 'Bachelor', 'occupation': 'Doctor',
     'relationship_goals': 'Marriage', 'lifestyle': 'Active'},

    {'age': 22, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Eldoret',
     'interests': ['marathon running', 'fitness', 'cooking'], 'education': 'Bachelor', 'occupation': 'Athlete',
     'relationship_goals': 'Casual', 'lifestyle': 'Active'},

    {'age': 35, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Nakuru',
     'interests': ['farming', 'politics', 'hiking'], 'education': 'PhD', 'occupation': 'Agricultural Economist',
     'relationship_goals': 'Long-term', 'lifestyle': 'Non-Smoker'},

    {'age': 26, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Thika',
     'interests': ['business', 'reading', 'coffee dates'], 'education': 'Bachelor', 'occupation': 'Entrepreneur',
     'relationship_goals': 'Long-term', 'lifestyle': 'Smoker'},

    {'age': 31, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Machakos',
     'interests': ['hiking', 'biking', 'camping'], 'education': 'Master', 'occupation': 'Civil Engineer',
     'relationship_goals': 'Marriage', 'lifestyle': 'Active'},

    {'age': 29, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Nyeri',
     'interests': ['cooking', 'gardening', 'travel'], 'education': 'Bachelor', 'occupation': 'Nurse',
     'relationship_goals': 'Marriage', 'lifestyle': 'Non-Smoker'},

    {'age': 40, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'Meru',
     'interests': ['farming', 'wildlife conservation', 'hiking'], 'education': 'Bachelor', 'occupation': 'Wildlife Ranger',
     'relationship_goals': 'Long-term', 'lifestyle': 'Active'},

    {'age': 25, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Kakamega',
     'interests': ['dance', 'music', 'social work'], 'education': 'Master', 'occupation': 'Social Worker',
     'relationship_goals': 'Casual', 'lifestyle': 'Non-Smoker'},
]


for p in sample_profiles:
    kb.add_profile(p)

matchmaker = Matchmaker(knowledge_base=kb)

age_input = widgets.IntSlider(description="Age", min=18, max=60, value=25)
gender_input = widgets.Dropdown(description="Gender", options=['Male', 'Female', 'Non-Binary'])
preferred_gender_input = widgets.Dropdown(description="Preferred Gender", options=['Male', 'Female', 'Any'])
location_input = widgets.Text(description="Location", placeholder="Enter city")
interests_input = widgets.Text(description="Interests (comma-separated)")
education_input = widgets.Dropdown(description="Education", options=['High School', 'Bachelor', 'Master', 'PhD'])
occupation_input = widgets.Text(description="Occupation")
relationship_goals_input = widgets.Dropdown(description="Goals", options=['Casual', 'Long-term', 'Marriage'])
lifestyle_input = widgets.Dropdown(description="Lifestyle", options=['Active', 'Non-Smoker', 'Smoker'])
submit_button = widgets.Button(description="Find Matches")
output_display = widgets.Output()

def find_matches(b):
    with output_display:
        clear_output()
        profile = {
            'age': age_input.value,
            'gender': gender_input.value,
            'preferred_gender': preferred_gender_input.value,
            'location': location_input.value,
            'interests': [i.strip() for i in interests_input.value.split(',') if i.strip()],
            'education': education_input.value,
            'occupation': occupation_input.value,
            'relationship_goals': relationship_goals_input.value,
            'lifestyle': lifestyle_input.value
        }
        user_id = kb.add_profile(profile)
        matches = matchmaker.find_matches(user_id, top_n=3)
        kb.profiles.pop(user_id)
        if not matches:
            print("No compatible matches found.")
        else:
            print("Top Matches:")
            for profile_id, score, details in matches:
                print(f"Profile {profile_id} - Compatibility Score: {score}")
                print(f"Matched Attributes: {', '.join(details)}")
                print("-" * 40)

submit_button.on_click(find_matches)
display(age_input, gender_input, preferred_gender_input, location_input, interests_input,
        education_input, occupation_input, relationship_goals_input, lifestyle_input, submit_button, output_display)


IntSlider(value=25, description='Age', max=60, min=18)

Dropdown(description='Gender', options=('Male', 'Female', 'Non-Binary'), value='Male')

Dropdown(description='Preferred Gender', options=('Male', 'Female', 'Any'), value='Male')

Text(value='', description='Location', placeholder='Enter city')

Text(value='', description='Interests (comma-separated)')

...

Text(value='', description='Occupation')

Dropdown(description='Goals', options=('Casual', 'Long-term', 'Marriage'), value='Casual')

Dropdown(description='Lifestyle', options=('Active', 'Non-Smoker', 'Smoker'), value='Active')

Button(description='Find Matches', style=ButtonStyle())

Output()

### First version to turn class code into a dating matchmaker KBS

In [ ]:
import random

class KnowledgeBase:
    """
    Stores and manages user profiles with unique identifiers.
    Acts as the knowledge repository for the matchmaking system.
    """

    def __init__(self):
        self.profiles = {}  # Dictionary to store profiles (profile_id: profile_data)
        self.next_id = 1  # Auto-incrementing ID counter

    def add_profile(self, profile_data):
        """
        Adds a new profile to the knowledge base with automatic ID assignment.
        :param profile_data: Dictionary containing profile attributes.
        :return: Assigned profile ID.
        """
        profile_id = self.next_id
        self.profiles[profile_id] = {'id': profile_id, **profile_data}
        self.next_id += 1
        return profile_id

    def get_profile(self, profile_id):
        """ Retrieves a profile by ID. """
        return self.profiles.get(profile_id)

    def get_all_profiles(self):
        """ Retrieves all profiles for matchmaking. """
        return list(self.profiles.values())


class Matchmaker:
    """
    Implements Tinder-like matchmaking using compatibility rules and weighted attributes.
    """

    def __init__(self, knowledge_base, weights=None, max_age_diff=10):
        """
        :param knowledge_base: KnowledgeBase instance.
        :param weights: Dictionary containing attribute weights for matchmaking (sum to 1).
        :param max_age_diff: Maximum age difference allowed between profiles.
        """
        self.knowledge_base = knowledge_base
        self.weights = weights or {
            'age': 0.2, 'interests': 0.3, 'location': 0.15,
            'education': 0.1, 'occupation': 0.1, 'lifestyle': 0.15
        }
        self.max_age_diff = max_age_diff

    def calculate_compatibility(self, profile_a, profile_b):
        """
        Computes weighted compatibility scores between two profiles.
        """

        # Check Gender Preference
        if profile_a['preferred_gender'] not in [profile_b['gender'], "Any"]:
            return 0  # No match if gender preference doesn't align

        if profile_b['preferred_gender'] not in [profile_a['gender'], "Any"]:
            return 0

        # Age Compatibility (Normalized inverse difference)
        age_diff = abs(profile_a['age'] - profile_b['age'])
        age_score = max(0, 1 - (age_diff / self.max_age_diff)) * self.weights['age']

        # Interest Compatibility (Jaccard similarity)
        interests_a = set(profile_a['interests'])
        interests_b = set(profile_b['interests'])
        intersection = len(interests_a & interests_b)
        union = len(interests_a | interests_b)
        jaccard_score = (intersection / union) * self.weights['interests'] if union > 0 else 0

        # Location Compatibility
        location_score = self.weights['location'] if profile_a['location'] == profile_b['location'] else 0

        # Education Compatibility
        education_score = self.weights['education'] if profile_a['education'] == profile_b['education'] else 0

        # Occupation Compatibility
        occupation_score = self.weights['occupation'] if profile_a['occupation'] == profile_b['occupation'] else 0

        # Lifestyle Compatibility (Exact Match)
        lifestyle_score = self.weights['lifestyle'] if profile_a['lifestyle'] == profile_b['lifestyle'] else 0

        # Final Weighted Score
        return age_score + jaccard_score + location_score + education_score + occupation_score + lifestyle_score

    def find_matches(self, user_id, top_n=5):
        """
        Finds the top N matches for a given user based on compatibility scores.
        """
        target_profile = self.knowledge_base.get_profile(user_id)
        if not target_profile:
            return []

        matches = []
        for profile in self.knowledge_base.get_all_profiles():
            if profile['id'] == user_id:
                continue  # Skip self-matching

            score = self.calculate_compatibility(target_profile, profile)
            if score > 0:  # Only include valid matches
                matches.append((profile['id'], round(score, 2)))

        # Sort matches by score and return top N
        matches.sort(key=lambda x: x[1], reverse=True)
        return matches[:top_n]


if __name__ == "__main__":
    # Initialize the Knowledge Base
    kb = KnowledgeBase()

    # Populate with sample profiles
    sample_profiles = [
        {'age': 25, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'New York',
         'interests': ['music', 'sports'], 'education': 'Bachelor', 'occupation': 'Engineer',
         'relationship_goals': 'Long-term', 'lifestyle': 'Active'},
        
        {'age': 28, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Los Angeles',
         'interests': ['music', 'travel'], 'education': 'Master', 'occupation': 'Designer',
         'relationship_goals': 'Casual', 'lifestyle': 'Active'},

        {'age': 32, 'gender': 'Male', 'preferred_gender': 'Female', 'location': 'New York',
         'interests': ['sports', 'travel'], 'education': 'PhD', 'occupation': 'Professor',
         'relationship_goals': 'Marriage', 'lifestyle': 'Non-Smoker'},

        {'age': 27, 'gender': 'Female', 'preferred_gender': 'Male', 'location': 'Chicago',
         'interests': ['music', 'sports'], 'education': 'Bachelor', 'occupation': 'Teacher',
         'relationship_goals': 'Long-term', 'lifestyle': 'Smoker'},
    ]

    for p in sample_profiles:
        kb.add_profile(p)

    # Initialize the Matchmaker
    matchmaker = Matchmaker(
        knowledge_base=kb,
        weights={'age': 0.2, 'interests': 0.3, 'location': 0.15, 'education': 0.1, 'occupation': 0.1, 'lifestyle': 0.15},
        max_age_diff=10
    )

    # Find matches for the first profile
    print("Top Matches for Profile 1:")
    for match in matchmaker.find_matches(1, top_n=3):
        print(f"Profile {match[0]} with Compatibility Score: {match[1]}")
